# Use the Model in the Real World

- Use data of GF satallite
- Test if out trained model (trained on massachusetts-roads-dataset) can do the task

In [ ]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
import torch
import numpy as np

from models import DLinkNet34, LinkNet34, FarSegNet

In [ ]:
FIG_SAVE_PATH = 'predict_result/mid_report/'

In [ ]:
def plt_improve(p):
    p.xticks([])
    p.yticks([])

In [ ]:
img_path = '/home/renyan/ossdata/gf-yunnan/IMG_0336.JPG'

In [ ]:
img = cv2.imread(img_path)
img_part = img[0:1500,0:1500] * 4

In [ ]:
transform = transforms.Compose([transforms.Resize([256, 256], 0), transforms.ToTensor(),])
img_part = transform(Image.fromarray(img))

In [ ]:
img_part.shape

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = LinkNet34.LinkNet34().to(device)
net.load_state_dict(torch.load('checkpoints/v2_linknet34_re_epoch20.pth'))

In [ ]:
plt.imshow(img_part.permute(1, 2, 0))
plt_improve(plt)
# plt.savefig(FIG_SAVE_PATH + 'img_gf_1.png', bbox_inches='tight')

In [ ]:
pred = net(img_part.unsqueeze(dim=0).to(device))
pred = (torch.sigmoid(pred).squeeze(0) > 0.51).type(torch.float32)
pred1 = torch.cat([pred]*3).permute(1, 2, 0)
plt.imshow(pred1.cpu())
plt_improve(plt)

In [ ]:
net = DLinkNet34.DLinkNet34().to(device)
net.load_state_dict(torch.load('checkpoints/v2_dlinknet34_re_epoch20.pth'))

In [ ]:
pred = net(img_part.unsqueeze(dim=0).to(device))
pred = (torch.sigmoid(pred).squeeze(0) > 0.51).type(torch.float32)
pred2 = torch.cat([pred]*3).permute(1, 2, 0)
plt.imshow(pred2.cpu())
plt_improve(plt)

## FarSeg

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = FarSegNet.FarSegNet().to(device)
net.eval()
net.load_state_dict(torch.load('checkpoints/' + 'v2_farseg_epoch100.pth'))

In [ ]:
img_part.unsqueeze(dim=0).to(device).shape

In [ ]:
probs, preds = net(img_part.unsqueeze(dim=0).to(device))

In [ ]:
pred = (probs > 0.2).float()
pred.shape

In [ ]:
plt.imshow(torch.cat([pred.squeeze().unsqueeze(dim=0)]*3).permute(1, 2, 0).cpu().detach().numpy())

In [ ]:
tmp = torch.cat([probs.squeeze().unsqueeze(dim=0)]*3).permute(1, 2, 0)

In [ ]:
tmp.shape

In [ ]:
# plt.hist(probs.flatten().detach().cpu().numpy())

In [ ]:
plt.imshow(probs.squeeze().cpu().detach().numpy()*5)
plt_improve(plt)
plt.savefig(FIG_SAVE_PATH + 'pred_gf_1_channel1.png', bbox_inches='tight')

In [ ]:
tmp = torch.cat([probs.squeeze(0)]*3).permute(1, 2, 0).detach().cpu().numpy()

In [ ]:
import utils
import matplotlib.pyplot as plt

plt.imshow(utils.change_tensor_to_plot(tmp))
plt_improve(plt)
# plt.savefig(FIG_SAVE_PATH + 'pred_gf_1_channel3.png', bbox_inches='tight')

In [ ]:
pred1.max()

In [ ]:
predtmp = (tmp > 0.3).astype(float)
kernel = np.ones((3,3), np.uint8)
erorsion_img = cv2.erode(predtmp, kernel, iterations=1)
plt.imshow(erorsion_img)
plt_improve(plt)

In [ ]:
pred1.shape